In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import dill
from tqdm import tqdm
from triangle import triangulate
from random import choice, sample
import sympy

### Load precomputed integrals

In [ ]:
poisson_finctional = dill.load(open('../calculations/hermite_cubic_matrix', 'rb'))
integral_values = dill.load(open('../calculations/hermite_cubic_right', 'rb'))

### Initialize the domain

In [ ]:
t = np.linspace(0, 2*np.pi, 50, endpoint=False)
points = np.stack((np.cos(t), np.sin(t))).T
points = np.array([
[0, 0],
[3, 0],
[3, 3],
[2, 3],
[2, 1],
[1, 1],
[1, 3],
[0, 3]
])
seg = np.array([[j, j+1] for j in range(points.shape[0]-1)] + [[points.shape[0] - 1, 0]])

### Run triangulation

In [ ]:
cndt = triangulate({'vertices':points, 'segments':seg}, opts='Rpq30a0.001neo1')
points = cndt['vertices']
triangles = cndt['triangles']
neighbours = cndt['neighbors']
vertice_maker_is_boundary = cndt["vertex_markers"]
print(f"Total number of points {points.shape[0]}")

### Visualize the triangulation

In [ ]:
all_colors = 100*list(mcolors.CSS4_COLORS.keys()) + list(mcolors.XKCD_COLORS.keys()) + list(mcolors.BASE_COLORS.keys())
colors = sample(all_colors, triangles.shape[0])

%matplotlib inline
fig, ax = plt.subplots(figsize=(5, 5))
ax.axis('equal')

for index, triangle in enumerate(tqdm(points[triangles])):
    t1 = plt.Polygon(triangle[:3], color=colors[index])
    fig.gca().add_patch(t1)
    x, y = triangle.mean(axis=0)
    
ax.set_xlim([points[:, 0].min(), points[:, 0].max()])
ax.set_ylim([points[:, 1].min(), points[:, 1].max()])

fig.tight_layout()
plt.show()

In [ ]:
x, y = sympy.symbols("x y")

In [ ]:
right_part_values = np.zeros((points.shape[0], 3))

In [ ]:
F = 1*sympy.Float(1)
F_x = F.diff(x)
F_y = F.diff(y)

In [ ]:
right_part_values[:, 0] = sympy.lambdify((x, y), F)(*points.T)
right_part_values[:, 1] = sympy.lambdify((x, y), F_x)(*points.T)
right_part_values[:, 2] = sympy.lambdify((x, y), F_y)(*points.T)

In [ ]:
matrix = np.zeros((3*points.shape[0], 3*points.shape[0]))
b = np.zeros(3*points.shape[0])

for idx, element in enumerate(tqdm(triangles)):
        
    x1, x2, x3 = points[element[0], 0], points[element[1], 0], points[element[2], 0]
    y1, y2, y3 = points[element[0], 1], points[element[1], 1], points[element[2], 1]
    
    delta = x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2
    
    P_1_x = (+y2 - y3) / delta
    P_1_y = (-x2 + x3) / delta
    
    P_2_x = (+y3 - y1) / delta
    P_2_y = (-x3 + x1) / delta
    
    arg = [P_1_x, P_1_y, P_2_x, P_2_y]
    right = right_part_values[element].flatten()
    
    FF = poisson_finctional(*arg, *right)
    ff = integral_values(*arg, *right)
    
    
    for local_point in range(3):

        if vertice_maker_is_boundary[element[local_point]] == True:
            
            matrix[3*element[local_point]+0, 3*element[local_point]+0] = 1
            
            for degree in range(1, 3):
                for other_point in range(3):
                    for other_degree in range(3):
                        matrix[3*element[local_point]+degree, 3*element[other_point]+other_degree] += \
                        2 * FF[3*local_point + degree, 3*other_point + other_degree]
            
            b[3*element[local_point]+0] = 0 
            b[3*element[local_point]+1] -= 2 * ff[3*local_point+1]
            b[3*element[local_point]+2] -= 2 * ff[3*local_point+2]
            
        else:
            for degree in range(3):
                for other_point in range(3):
                    for other_degree in range(3):
                        matrix[3*element[local_point]+degree, 3*element[other_point]+other_degree] += \
                        2 * FF[3*local_point + degree, 3*other_point + other_degree]
            
            b[3*element[local_point]+0] -= 2 * ff[3*local_point+0]
            b[3*element[local_point]+1] -= 2 * ff[3*local_point+1]
            b[3*element[local_point]+2] -= 2 * ff[3*local_point+2]

In [ ]:
c = np.linalg.solve(matrix, b)

In [ ]:
%matplotlib widget
fig = plt.figure(figsize=(9, 9))
ax = plt.axes(projection='3d')
orig = (points[:, 0]**2+points[:, 1]**2-1)/4

F_approx = c[0::3]
F_x_approx = c[1::3]
F_y_approx = c[2::3]


ax.plot_trisurf(points[:, 0], points[:, 1], F_approx,
                cmap='magma',
                triangles=triangles[:, :3], 
)
plt.show()